### 1: STRUCTURE

In human genetics, a common task is to collect [genotype data](https://en.wikipedia.org/wiki/Genotype#Genotyping) from a collection of individuals, and study this data in order to understand what population(s) the individuals belong to. The program [STRUCTURE](https://academic.oup.com/genetics/article/155/2/945/6048111) (Pritchard, Stephens & Donnelly, 2003) is an [influential method](https://academic.oup.com/genetics/article/204/2/391/6072054) for doing this. 

Formally, given a matrix $G \in \{0,1\}^{N \times L \times 2}$ of diploid genotype data from $N$ individuals type at $L$ loci, and some estimate $K$ of the number of populations, STRUCTURE assumes that $G$ is generated according to the following probabilistic model:

$$
\begin{align}
G_{n\ell a} & \sim \mathrm{Binom}(1,P_{Z_{n\ell}\ell})\\
P_{k \ell} & \sim \mathrm{Beta}(fp,f(1-p)) \\
Z_{n\ell a} &\sim \mathrm{Categorical}(Q_n) \\
Q_n &\sim \mathrm{Dirichlet}(\alpha) \in \Delta^{K-1}
\end{align} 
$$
Here $F,p\in[0,1]$ and $\alpha \in \mathbb{R}_{+}^{K}$ are hyperparameters.

This model is closely related to the latent Dirichlet allocation method of for document topic modeling. 

Here is a pure Python implementation of a Gibbs sampler for this model.

In [ ]:
import numpy as np

def gibbs_sampler(G, K, rng, num_iterations=1000, alpha=1.0, F=.1, p=.1):
    """
    Run Gibbs sampler for the STRUCTURE model.
    
    data: N x L x 2 array of genotype data
    K: Number of populations
    rng: Random number generator.
    num_iterations: Number of Gibbs sampling iterations
    alpha: Dirichlet prior parameter for q
    F: estimate of Fst between populations.
    p: mean allele frequency
    """
    N, L, _ = G.shape
    assert G.shape == (N, L, 2)
    # Initialize parameters
    Q = rng.dirichlet([alpha] * K, size=N)
    f = (1 - F) / F
    f, p = [np.broadcast_to(x, [K, L]) for x in [f, p]] 
    a_prior = f * p
    b_prior = f * (1 - p)
    P = rng.beta(a_prior, b_prior)   # balding-nichols model
    P = np.stack([P, 1 - P], axis=2)
    # Initialize z assignments randomly
    Z = rng.integers(0, K, size=(N, L, 2))
    for iteration in range(num_iterations):
        # Update z
        for i in range(N):
            for l in range(L):
                for j in range(2):
                    probs = Q[i] * P[:, l, G[i, l, j]]
                    probs /= probs.sum()
                    Z[i, l, j] = rng.choice(K, p=probs)
        # Update p
        for k in range(K):
            for l in range(L):
                counts = np.zeros([2])
                idx = np.where(Z[:,l,:] == k)
                alleles = G[:,l,:][idx]
                for a in range(2):
                    counts[a] = np.sum(alleles == a)
                P[k, l] = rng.beta(counts[0] + a_prior[k, l], counts[1] + b_prior[k, l])
        # Update q
        for i in range(N):
            counts = np.zeros(K)
            for k in range(K):
                counts[k] = np.sum(Z[i,:,:] == k)
            Q[i] = rng.dirichlet(counts + alpha)
        if iteration % 100 == 0:
            print(f"Iteration {iteration}")
    return Q, P

1. Rewrite the above sampler using any of the techniques we have learned in class so far (vectorization, Numba, Jax, Cuda/GPU, Cython, multithreading/multiprocessing) in order to get a speedup.
2. Report the speedup you obtained using benchmarking.
3. Explain why your code is faster.


Test your implementation using the following code, which simulates a model with three populations and returns a genotype matrix:

In [ ]:
import stdpopsim
species = stdpopsim.get_species("HomSap")
model = species.get_demographic_model("OutOfAfrica_3G09")
chrom = species.get_contig('1', length_multiplier=0.01)
samples = model.get_samples(10, 10, 10)
engine = stdpopsim.get_engine("msprime")
ts = engine.simulate(model, chrom, samples)

G = ts.genotype_matrix().T
# convert diploid to tensor
G = np.array([[0, 0], [0, 1], [1, 1]])[G]

In [ ]:
rng = np.random.default_rng()
p = G.mean((0, 2))
F = ts.Fst([np.arange(10), np.arange(10, 20)])
gibbs_sampler(G, 3, rng, p=p, F=F)

### 2. Composite likelihood

[Composite likelihood](https://www3.stat.sinica.edu.tw/sstest/oldpdf/A21n11.pdf) is a technique for performing approximate likelihood-based inference in models where the full likelihood is intractable. The basic idea is replace the overall likelihood with a "product of marginals" which can be tractably computed. 

For example, consider the spatial 2D Gaussian process model 

$$y(s) = \mu + \varepsilon(s),$$

where $s=(x,y)$, $y(s)$ is the observed value at location $s$, and the error term is zero-mean Gaussian process with covariance function 

$$\mathrm{cov}(\varepsilon(s_i), \varepsilon(s_j)) = \sigma^2 \exp \left(-\frac{\| s_i - s_j \|}{\phi} \right)$$

for variance parameter $\sigma^2>0$ and range parameter $\phi>0$.


1. Suppose you are given a $10^8$-dimensional sample $\mathbf{y}=(y(s_1),\dots,y(s_{10^7}))$ from the GP.
   
   a. Explain why estimating $\mu$ is easy. (Explain why we can assume $\mu=0$.)
   
   b. Explain why estimating $\sigma^2$ and $\phi$ using e.g. naive maximum likelihood is challenging.

2. Consider the composite likelihood function 

    $$\ell_c(\mathbf{y} \mid \theta) = \sum_{i,j \in S} \log \mathbb{P}(y(s_{i}), y(s_{j}) \mid \theta),$$

    where $\theta=(\sigma^2,\phi)$, and $S\subset [n]^2$ is some subset of indices. Here $\mathbb{P}(y(s_{i}), y(s_{j}) \mid \theta)$ denotes the marginal distribution of $y$ at the two points $s_i$ and $s_i$. By the definition of GP, it is bivariate normal with covariance shown above.

    a. Explain why computing $\ell_c$ is easier (especially if you are smart about choosing $S$.)

    b. Argue that maximizing $\ell_c$ over $\theta$ results in an unbiased estimator.

    c. Explain intuitively (no math needed) why applying standard asymptotic theory to the resulting MCLE (maximum *composite* likelihood estimator) would result in the wrong covariance matrix. Would confidence intervals tend to be too narrow or too broad? Why? (Hint: consider the "effective sample size".)
    
3. The correct asymptotic covariance matrix for the MCLE is called the Godambe Information Matrix. It is given by

    $$G = H^{-1} J H^{-1}$$

    where $H = -\nabla^2_\theta \ell_c(\hat{\theta})$ is the observed information, $U_c(\theta)=\nabla_\theta \ell_c(\theta)$ is the score function, and $J$ is an estimate (sample covariance) of $\mathrm{cov}\, U_c(\hat{\theta}).$ 
    
    (Notice that $J=H$ for the usual likelihood function -- this is called [Bartlett's second identity](https://math.stackexchange.com/questions/2026428/what-is-second-bartlett-identity) and is the reason why the Fisher information $F=J=H$.)

    a. Perform a small simulation study which estimates the covariance of $\hat{\theta}$ when $\theta$ maximizes a composite likelihood.

    b. Use automatic differentiation to compute $G$ for your choice of composite likelihood function, as well as $F$, the observed Fisher information.

    c. Compare the results of part a) with $F$ and $G$. Does $F$ under/overestimate asymptotic variance and if so by how much?